In [ ]:
import numpy as np

In [ ]:
#@title misc.py
import sys
import numpy as np

def legalMove(board, moveLoc):
    BOARD_SIZE = board.shape[0]
    if moveLoc[0] < 0 or moveLoc[0] >= BOARD_SIZE or \
       moveLoc[1] < 0 or moveLoc[1] >= BOARD_SIZE: 
        return False

    if board[moveLoc] == 0:
        return True
    return False

def rowTest(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    mask = np.ones(X_IN_A_LINE, dtype=int)*playerID

    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE-X_IN_A_LINE+1):
            flag = True
            for i in range(X_IN_A_LINE):
                if board[r,c+i] != playerID:
                    flag = False
                    break
            if flag:
                return True

    return False        

def diagTest(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    for r in range(BOARD_SIZE - X_IN_A_LINE + 1):
        for c in range(BOARD_SIZE - X_IN_A_LINE + 1):
            flag = True
            for i in range(X_IN_A_LINE):
                if board[r+i,c+i] != playerID:
                    flag = False
                    break
            if flag:
                return True
    return False

def winningTest(playerID, board, X_IN_A_LINE):  
    if rowTest(playerID, board, X_IN_A_LINE) or diagTest(playerID, board, X_IN_A_LINE):
        return True

    boardPrime = np.rot90(board)
    if rowTest(playerID, boardPrime, X_IN_A_LINE) or diagTest(playerID, boardPrime, X_IN_A_LINE):
        return True
    
    return False


The board look like:

[[-1,0,-1,1,0]

[-1,0,-1,1,0]

[-1,0,-1,1,0]

[-1,0,-1,1,0]

[-1,0,-1,1,0]]

which -1/1 is the player ID(self.ID)

In [ ]:
BOARD_SIZE = 11   # size of the board is 11-by-11
X_IN_A_LINE = 5   # play the standard game with 5 stones in a line
TIME_OUT = 5      # player must return a move within 5 seconds


In [ ]:
#@title gomokuAgent init
class GomokuAgent:


  #ID: Player ID assigned by the game platform in a game, either 1 or -1
  #boardSize:  length of the board, an integer
  #XINALINE: winning condition, an integer

  def __init__(self, ID, BOARD_SIZE, X_IN_A_LINE):
        self.ID = ID
        self.BOARD_SIZE = BOARD_SIZE
        self.X_IN_A_LINE = X_IN_A_LINE


In [ ]:
#@title Random player class (From gomokuAgentRand)
class Player(GomokuAgent):
    def move(self, board):
        while True:
            moveLoc = tuple(np.random.randint(self.BOARD_SIZE, size=2))
            if legalMove(board, moveLoc):
                return moveLoc


In [ ]:
#@title take a turn (from gomoku.py)
import sys, time, signal
import numpy as np
import os

import concurrent.futures

from time import time
from random import randint


# an empty class to host the time-out exception
class TimeOutException(Exception):
    pass

# handler for time out
def handler(signum, frame):
    print("Player timeout - signal -")
    raise TimeOutException()

"""
CHANGE:
Added turn_id to the turn function to replace the player.ID calls
This protects against ID spoofing
"""
# turn taking function
def turn(board, player, turn_id):

    # make a copy of the board, which is passed to the agent
    tempBoard = np.array(board)

    # TIME_OUT seconds Timer
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            playerThread = executor.submit(player.move, tempBoard)
            moveLoc = playerThread.result(TIME_OUT+1)
    except TimeOutException:        
        # Caught the signal timeout exception
        print("before pass")
        pass
    except concurrent.futures.TimeoutError:
        print("Player" + str(turn_id) + " time out.")
        return turn_id*1, board        

    # test if the move is legal - on the original board
    if legalMove(board, moveLoc):
        board[moveLoc] = player.ID
    else:
        print("Player " + str(player.ID) + " illegal move at " + str(moveLoc))
        return turn_id*-1, board

    # test if any player wins the game
    if winningTest(player.ID, board, X_IN_A_LINE):
        return turn_id, board

    # move to the next turn
    return 0, board


ab-pruning

To see if a there is any way to win in this chess

Search Depth

Branching Factor  ** Branching factor control **

Evaluation Process****

# Implenmentation

## Huristic 
The winning proberbility of a board can be like:
1. Density of chess in certain area
2. Pattern of chess (Can do it by kunnel?)

**Implenmentation**
the closer to 0, the more likely this board can win(0 is end game)

Reference:

[Gomoku Steragy](https://www.wikihow.com/Play-Gomoku#:~:text=Aim%20for%205%20pieces%20in%20a%20row%20to%20win%20the%20game.&text=The%20line%20can%20go%20in,overlines%E2%80%9D%20and%20do%20not%20count.)

Useful for get_children_move():
[Game theory](http://gomokuworld.com/gomoku/1)

In [ ]:
#@title blocking 4 (abandoned)
# rotate not correct
def rotate_coor(coors,board_size):
  a=np.zeros((board_size,board_size))
  for x,y in coors:
    a[x][y]=1
  a=np.rot90(a,k=3)
  a=tuple(map(tuple,np.argwhere(a==1)))
  #print(a)
  if len(coors)==1:
    a=a[0]
  return a  

def row4Test(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    mask=np.ones((X_IN_A_LINE,X_IN_A_LINE))*playerID
    np.fill_diagonal(mask,0)
                     
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE-X_IN_A_LINE+1):
            for m in range(mask.shape[0]):
                if (board[r,range(c,c+X_IN_A_LINE)] == mask[m]).all():
                  #print(f"coor:{r},{c} m:{m} ")
                  return True,(r,c+m)

    return False,0        

def diag4Test(playerID, board, X_IN_A_LINE):
    mask=np.ones((X_IN_A_LINE,X_IN_A_LINE))*playerID
    np.fill_diagonal(mask,0)
    #print(mask)
    BOARD_SIZE = board.shape[0]
    for r in range(BOARD_SIZE - X_IN_A_LINE + 1):
        for c in range(BOARD_SIZE - X_IN_A_LINE + 1):
            for m in range(mask.shape[0]):
                ranges=list(map(list,zip(range(r,r+X_IN_A_LINE),range(c,c+X_IN_A_LINE))))
                testarr=[board[i][j] for i,j in ranges]
                #print(f"coor:{r},{c} testarr:{testarr} ranges:{ranges} mask:{mask[m]}")
                if (testarr==mask[m]).all():
                  return True,(r+m,c+m)
    return False,0

def blocking4Test(playerID, board, X_IN_A_LINE):
    t,a = row4Test(playerID, board, X_IN_A_LINE)
    if t:
      return True,a
    t,a = diag4Test(playerID, board, X_IN_A_LINE)
    if t:
      return True,a

    boardPrime = np.rot90(board)
    #print(boardPrime)
    t,a = row4Test(playerID, boardPrime, X_IN_A_LINE)
    if t:
      return True,rotate_coor([a],board.shape[0])

    t,a = diag4Test(playerID, boardPrime, X_IN_A_LINE)
    if t:
      return True,rotate_coor([a],board.shape[0])
    
    return False,0

In [ ]:
#@title Blocking 4 2 (Improve performance ver)
def row4Test2(playerID, board, X_IN_A_LINE,mask,r,c):
    for m in range(mask.shape[0]):
      if (board[r,range(c,c+X_IN_A_LINE)] == mask[m]).all():
        #print(f"coor:{r},{c} m:{m} ")
        return True,(r,c+m)
    return False,0        

def diag4Test2(playerID, board, X_IN_A_LINE,mask,r,c):
    #print(mask)
    for m in range(mask.shape[0]):
                ranges=list(map(list,zip(range(r,r+X_IN_A_LINE),range(c,c+X_IN_A_LINE))))
                testarr=[board[i][j] for i,j in ranges]
                #print(f"coor:{r},{c} testarr:{testarr} ranges:{ranges} mask:{mask[m]}")
                if (testarr==mask[m]).all():
                  return True,(r+m,c+m)
    return False,0

def blocking4Test(playerID, board, X_IN_A_LINE):
  mask=np.ones((X_IN_A_LINE,X_IN_A_LINE))*playerID
  np.fill_diagonal(mask,0)
  boardPrime = np.rot90(board)
  BOARD_SIZE = board.shape[0]
  t,a=False,0
  #print(BOARD_SIZE)
  for r in range(BOARD_SIZE):
    for c in range(BOARD_SIZE-X_IN_A_LINE+1):
      t,a = row4Test2(playerID, board, X_IN_A_LINE,mask,r,c)
      if t:
        return True,a

      t,a = row4Test2(playerID, boardPrime, X_IN_A_LINE,mask,r,c)
      if t:
        return True,rotate_coor([a],BOARD_SIZE)

  for r in range(BOARD_SIZE - X_IN_A_LINE + 1):
    for c in range(BOARD_SIZE - X_IN_A_LINE + 1):
    
      t,a = diag4Test2(playerID, board, X_IN_A_LINE,mask,r,c)
      if t:
        return True,a

      t,a = diag4Test2(playerID, boardPrime, X_IN_A_LINE,mask,r,c)
      if t:
          return True,rotate_coor([a],board.shape[0])
  
  return False,0

In [ ]:
# to identifly blocking 3 line and possible child to block the opponent move
#@title Open 3

# rotate not correct

def rotate_coors(coors,board_size):
  a=np.zeros((board_size,board_size))
  for x,y in coors:
    a[x][y]=1
  a=np.rot90(a,k=3)
  a=tuple(map(tuple,np.argwhere(a==1)))
  #print(a)
  return a

def row3Test(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    mask=np.ones((X_IN_A_LINE,X_IN_A_LINE))*playerID
    np.fill_diagonal(mask,0)
    #print(mask)
    children = []                 
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE-X_IN_A_LINE+1):
            for m in range(mask.shape[0]):
                if (board[r,range(c,c+X_IN_A_LINE)] == mask[m]).all():
                  #print(f"coor:{r},{c} m:{m} ")
                  children.append((r,c+m))

    return children

def diag3Test(playerID, board, X_IN_A_LINE):
    mask=np.ones((X_IN_A_LINE,X_IN_A_LINE))*playerID
    np.fill_diagonal(mask,0)
    children = []
    #print(mask)
    BOARD_SIZE = board.shape[0]
    for r in range(BOARD_SIZE - X_IN_A_LINE + 1):
        for c in range(BOARD_SIZE - X_IN_A_LINE + 1):
            for m in range(mask.shape[0]):
                ranges=list(map(list,zip(range(r,r+X_IN_A_LINE),range(c,c+X_IN_A_LINE))))
                testarr=[board[i][j] for i,j in ranges]
                #print(f"coor:{r},{c} testarr:{testarr} ranges:{ranges} mask:{mask[m]}")
                if (testarr==mask[m]).all(): 
                    children.append((r+m,c+m))
    return children

def open3(playerID, board, X_IN_A_LINE):
    children =[]
    children.extend(row3Test(playerID, board, X_IN_A_LINE-1))
    children.extend(diag3Test(playerID, board, X_IN_A_LINE-1))
    
    boardPrime = np.rot90(board)
    children.extend(rotate_coors(row3Test(playerID, boardPrime, X_IN_A_LINE-1),board.shape[0]))
    children.extend(rotate_coors(diag3Test(playerID, boardPrime, X_IN_A_LINE-1),board.shape[0]))

    return np.array(children)



In [ ]:
#@title Open 3 (Improve perfformance)

def row3Test2(playerID, board, X_IN_A_LINE,mask,r,c):
    children=[]
    for m in range(mask.shape[0]):
                #print(f"coor:{r},{c} m:{m} ")
                #print(f"{board[r,range(c,c+X_IN_A_LINE)]},{mask[m]}")
                if (board[r,range(c,c+X_IN_A_LINE)] == mask[m]).all():
                  children.append((r,c+m))

    return children

def diag3Test2(playerID, board, X_IN_A_LINE,mask,r,c):
    children=[]

    for m in range(mask.shape[0]):
                ranges=list(map(list,zip(range(r,r+X_IN_A_LINE),range(c,c+X_IN_A_LINE))))
                testarr=[board[i][j] for i,j in ranges]
                #print(f"coor:{r},{c} testarr:{testarr} ranges:{ranges} mask:{mask[m]}")
          
                if (testarr==mask[m]).all(): 
                    children.append((r+m,c+m))
    return children

def open3(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    mask=np.ones((X_IN_A_LINE-1,X_IN_A_LINE-1))*playerID
    np.fill_diagonal(mask,0)
    boardPrime = np.rot90(board)

    #print(mask)
    children = []
    children_need_rotate=[]                 
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE-X_IN_A_LINE+2):
          children.extend(row3Test2(playerID, board, X_IN_A_LINE-1,mask,r,c))
          children_need_rotate.extend(row3Test2(playerID, boardPrime, X_IN_A_LINE-1,mask,r,c))

    for r in range(BOARD_SIZE - X_IN_A_LINE + 2):
        for c in range(BOARD_SIZE - X_IN_A_LINE + 2):
          children.extend(diag3Test2(playerID, board, X_IN_A_LINE-1,mask,r,c))
          children_need_rotate.extend(diag3Test2(playerID, boardPrime, X_IN_A_LINE-1,mask,r,c))
    children.extend(rotate_coors(children_need_rotate,BOARD_SIZE))
    return np.array(children)


In [ ]:
#@title Difference of open3
#@title Open3Diff (Improve perfformance)

def row3Test3(playerID, board, X_IN_A_LINE,mask,r,c):
    for m in range(mask.shape[0]):

                #print(f"coor:{r},{c} m:{m} ")
      #print(f"{board[r,range(c,c+X_IN_A_LINE)]},{mask[m]}")
      if (board[r,range(c,c+X_IN_A_LINE)] == mask[m]).all():
        #print("r1")
        return 1
      if (board[r,range(c,c+X_IN_A_LINE)] == (-mask[m])).all():
        #print("-r1")
        return -1
    return 0

def diag3Test3(playerID, board, X_IN_A_LINE,mask,r,c):
    for m in range(mask.shape[0]):
      ranges=list(map(list,zip(range(r,r+X_IN_A_LINE),range(c,c+X_IN_A_LINE))))
      testarr=[board[i][j] for i,j in ranges]
                #print(f"coor:{r},{c} testarr:{testarr} ranges:{ranges} mask:{mask[m]}")
      if (testarr==(mask[m])).all(): 
        #print("d1")
        return 1
      if (testarr==(-mask[m])).all(): 
        #print("-d1")
        return -1
    return 0

def open3diff(playerID, board, X_IN_A_LINE):
    BOARD_SIZE = board.shape[0]
    mask=np.ones((X_IN_A_LINE-1,X_IN_A_LINE-1))*playerID
    np.fill_diagonal(mask,0)
    boardPrime = np.rot90(board)
    #print(boardPrime)
    diff = 0               
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE-X_IN_A_LINE+2):
          diff+=(row3Test3(playerID, boardPrime, X_IN_A_LINE-1,mask,r,c)
                +row3Test3(playerID, board, X_IN_A_LINE-1,mask,r,c))
          pass
    for r in range(BOARD_SIZE - X_IN_A_LINE + 2):
        for c in range(BOARD_SIZE - X_IN_A_LINE + 2):
          diff+=(diag3Test3(playerID, board, X_IN_A_LINE-1,mask,r,c)
                +diag3Test3(playerID, boardPrime, X_IN_A_LINE-1,mask,r,c))
    
    return diff


In [ ]:
board2=np.array([[-1 , 1 ,-1 , 0,0],
                [ 0,  0  ,-1,  -1,0],
                [ 0 ,0  ,0 , 0,0],
                 [0  ,-1 , 0  ,1,0],
                 [ -1  ,-1 , 0  ,1,0]])
print(blocking4Test(-1,board2,4))

(True, (2, 2))


In [ ]:
board2=np.array([[-1 , 1 ,-1 , 0,0],
                [ -1,  0  ,-1,  0,0],
                 [ 0 ,0  ,0 , 0,0],
                 [0  ,-1 , 0  ,1,0],
                 [ 0  ,1 , 0  ,1,0]])
board4=np.array([[ 1 , 1 , 1, 0 , 0 , 0 , 0  ,0  ,0  ,0 , 0],
 [ 0 , 0  ,0 , 0  ,0,  0  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0 , 0 ,-1 , 0  ,0 , 0 , 0  ,0 ,-1],
 [ 0 , 0  ,0 , 0  ,0 , 0  ,0 , 0,  0,  0 ,-1],
 [ 0 , 0  ,0  ,0  ,0 , 0  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 1  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 0  ,0 , 0  ,0 , 0 , 0],
 [ 0  ,0  ,0  ,0 , 0 , 0 , 0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0, 0 , 0,  0 , 0 ,-1 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 0 , 0 , 0 ,-1 , 0 , 0],
 [ 0 , 0 , 0  ,0,  0 , 0 ,-1  ,0  ,0  ,0  ,0]])
print(open3(1,board4,5))


[[0 3]]


In [ ]:
board2=np.array([[0 , 0 , 0, 0,0],
                [ 0,  0  ,0,  0,0],
                 [ 0 ,0  ,0 , 0,0],
                 [0  ,0 , 0  , 1,0],
                 [ 0  ,0 , 0  ,1,1]])
print(open3diff(1,board2,4))

3


In [ ]:
def huristic(self,board):
  #Here is just predict one more step...
  if False:
    if blocking4Test(self.ID,board,self.X_IN_A_LINE)[0]:
      return 1
    if blocking4Test(-self.ID,board,self.X_IN_A_LINE)[0]:
      return -1
  #How to normalize the distibution a gaussian distribution
  
  return open3diff(self.ID,board,self.X_IN_A_LINE)/(board.shape[0]**2)

GomokuAgent.huristic=huristic

In [ ]:
p = GomokuAgent(1, 5, 4)

board2=np.array([[-1 , 0 ,0 , 0,0],
                [ 0,  -1  ,0, -1 ,0],
                [ 0 ,0 ,0, 0,0],
                 [0  ,0 , 0  ,1,0],
                 [ 0  ,0 , 0  ,1,0]])
print(p.huristic(board2))

-0.04


This is better to get order from largest prob to lower prob to win. Have a primary winning huristic guess

In [ ]:
def get_children_move(self,board,max_children=3):
    def delete_occurancce(a,b):
        #print(f"a{a}\n,b{b}")
        flattenb=[np.array2string(i.flatten()) for i in b]
        #print(f"flattenb{flattenb}")
        flattena=[np.array2string(i.flatten()) for i in a]
        #print(f"flattena{flattena}")
        ind=[flattena.index(b) for b in flattenb]
        #print(f"ind{ind}")
        # if all node are visited before, go back to parent node and go to another child
        newa=np.delete(a,ind,axis=0)
        return newa


    #identifly opponent open 3 and block
    inds=open3(self.ID,board,self.X_IN_A_LINE)
    newarr=np.argwhere(board==0)[:max_children]
    #print(inds)
    if inds.size>0:
      if inds.size==newarr.size:
        return inds

      newarr=delete_occurancce(np.argwhere(board==0),inds)

      newarr=np.concatenate((inds,newarr))
    #print(newarr)
    return newarr

GomokuAgent.get_children_move=get_children_move

In [ ]:
p = GomokuAgent(-1, BOARD_SIZE, X_IN_A_LINE)

a=np.array([[ 0 , 0 , 0 ,-1],
 [ 0 , 0  ,0  ,0],
 [ 0  ,0 , 1  ,0],
 [ 0 , 0 , 0 , 0]])

print(p.get_children_move(a))

[[0 0]
 [0 1]
 [0 2]]


## Utility

In [ ]:
def next_board(self,move,board,ID):
    newboard=np.copy(board)
    newboard[move[0]][move[1]]=ID
    return newboard
  
def children_board_factory(self,parent,ID,max_children=3):
    children_move=self.get_children_move(parent,max_children)
    children_boards=np.array([self.next_board(move,parent,ID) for move in children_move])
    return children_boards

GomokuAgent.next_board=next_board
GomokuAgent.children_board_factory=children_board_factory

In [ ]:
def end_game(self,board): # The closer to 1, the closer to win
    if winningTest(self.ID, board, X_IN_A_LINE ): #Win
      #print(f"win{board}")
      return True , 1 
    elif winningTest(-self.ID, board, X_IN_A_LINE): #Lose
      return True , -1
    if not (0 in board) : #Draw
      return True , 0
    else: return False , self.huristic(board) # reach max depth

GomokuAgent.end_game=end_game

In [ ]:
#@title ab_purning (comment out ab part)
def ab_pruning(self,board,m_depth=np.inf):

      def maximization(board,alpha,beta,max_depth):
        end, h = self.end_game(board)
        if end or max_depth==0:
          #print(f"maxHit{h} \n board:{board}")
          #mutiply by max_depth becuse the earier the winning is better
          return h*max_depth,alpha,beta #Utility

        vmax=-np.inf
        b=board
        children=self.children_board_factory(board,self.ID)
        #print(f"max board{children}")
        for child in children:
          va,_,beta=minimization(b,alpha,beta,max_depth-1)
          #print(f"Minimization{va}")
          vmax = max(va,vmax)
          alpha=max(alpha,vmax)
          #purning
          if alpha>=beta:
              #As we know this branch do not evaluate anymore,
              break
#          alpha = max(alpha,vmax)

        return vmax,alpha,alpha

      def minimization(board,alpha,beta,max_depth):
        end, h = self.end_game(board)
        if end or max_depth==0:
          #print(f"minHit{h}\n board:{board}")

          return h*max_depth,alpha,beta #Utility

        vmin=np.inf
        b=board
        children=self.children_board_factory(b,-self.ID)
        #print(f"min board{children}")

        for child in children:
          vb,alpha,_=maximization(child,alpha,beta,max_depth-1)
          #print(f"Minimization{vb}")
          vmin = min(vb,vmin)
          beta = min(beta,vmin)
         
          if alpha>=beta:
              break
#          if vmin>=alpha:
#              #print(f"purnMinimization{vmin}")
#              return vmin,alpha,beta
#         alpha = min(alpha,vmin)
        return vmin,beta,beta

      v=-np.inf
      alpha=-np.inf
      beta=+np.inf
      children_move=self.get_children_move(board)
      children_boards=np.array([self.next_board(child,board,self.ID) for child in children_move])
      best=0
      #maximization
      for i in range(children_boards.shape[0]):
         vp,_,beta=minimization(children_boards[i],alpha,beta,m_depth-1)
         #print(f"turn:{children_move[i]}\n vp:{vp}\n v={v}")
         if vp>v:
           best=i
           #print(f"best{best}")
           v=vp
         
           alpha=max(alpha,vp)
          #purning
         #if alpha>beta:
            #print(f"purn!{i}")
            #break
#         if v<beta:
#            #print(f"Purning:{children_move[best]}\n v:{v}\n beta:{neta}")
#            return children_move[best]
          
#         alpha = max(alpha,v)

      #print(children_move[best])
      return children_move[best]

GomokuAgent.ab_pruning=ab_pruning


board: a 11-by-11 numpy ndarray representing the game board
returns: a tuple with two elements representing the coordinate of the move


while True:
          moveLoc = tuple(np.random.randint(self.BOARD_SIZE, size=2))
            if legalMove(board, moveLoc):
                return moveLoc

In [ ]:
def move(self, board):
  # if this is the first move, then hardcode it to the middle
  if self.ID==1 and np.array_equal(board, np.zeros(board.shape)):
    return (int(board.shape[0]/2),int(board.shape[0]/2))

  #If we are going to win, immediatly win:
  t,a = blocking4Test(self.ID,board,X_IN_A_LINE)
  if t:
    return a
  # if opponent is going to win, block immediately
  t,a = blocking4Test(-self.ID,board,X_IN_A_LINE)
  if t:
    return a

  return tuple(self.ab_pruning(board,3))

GomokuAgent.move=move

In [ ]:
p = GomokuAgent(1, BOARD_SIZE, X_IN_A_LINE)
board=np.array([[-1,1],[0,1]])
board2=np.array([[ 1,  -1 , 1],
                  [ 0 , 0 , 0],
                  [ 0, 0 , 0]])
board3=np.array([[ 1 , 1 , 1, -1],
 [ 1, -1,  1 , 0],
 [ 0 ,-1 , 1, -1],
 [ 0, -1  ,0 , 0]])

board4=np.array([[ 1 , 1 , 1, 0 , 0 , 0 , 0  ,0  ,0  ,0 , 0],
 [ 0 , 0  ,0 , 0  ,0,  0  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0 , 0 ,-1 , 0  ,0 , 0 , 0  ,0 ,-1],
 [ 0 , 0  ,0 , 0  ,0 , 0  ,0 , 0,  0,  0 ,-1],
 [ 0 , 0  ,0  ,0  ,0 , 0  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 1  ,0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 0  ,0 , 0  ,0 , 0 , 0],
 [ 0  ,0  ,0  ,0 , 0 , 0 , 0 , 0 , 0 , 0 , 0],
 [ 0 , 0  ,0, 0 , 0,  0 , 0 ,-1 , 0 , 0 , 0],
 [ 0 , 0  ,0  ,0 , 0 , 0 , 0 , 0 ,-1 , 0 , 0],
 [ 0 , 0 , 0  ,0,  0 , 0 ,-1  ,0  ,0  ,0  ,0]])
print(p.move(board4))


(0, 3)


In [ ]:
#@title Run this cell to test (from gomoku.py)
player1 = GomokuAgent(1, BOARD_SIZE, X_IN_A_LINE)
player2 = Player(-1, BOARD_SIZE, X_IN_A_LINE)


    # initialize the board
board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)

    # connect the alarm signal with the handler
signal.signal(signal.SIGALRM, handler)

    # play the game
winner = 0
while True:
        end = False
        """
        CAHNGE:
        Added turn_id to the loop, this keeps the turn id separate from the 
        <GomokuAgent>.ID which is accessable and writeable from inside GomokuAgent
        """
        for player, turn_id in [(player1, 1), (player2, -1)]:
            id, board = turn(board, player, turn_id)
            print(board)
            """
            CHANGE:
            Move draw check to inside play loop. A draw will always be decided after player 1's turn and so
            insisting player 2 must always make a move after player 1 will lead to a loss for player 2 where
            a draw was possible
            """
            if not 0 in board:
                print("Draw.")
                end = True
                break
            if id != 0:
                print("Winner: " + str(id))
                end = True
                break
        if end:
            break        


[[0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]
[[ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]]
[[ 1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 

In [ ]:
board2=np.array([[1 ,0 ,0],
 [ 0 , 1  ,0],
 [ 0  ,1 , 1]])
r=1
c=2
d=3
[r,range(c,c+d)]
ranges=list(map(list,zip(range(r,d),range(r,d))))
print(ranges)
a=[*[1,2,3,4]]
print(a)

[[1, 1], [2, 2]]
[1, 2, 3, 4]


In [ ]:
!git config --global user.email “appleno0610@gmail.com”
!git config --global user.name “HarleyAppleChoi”

error: open("drive/MyDrive/ Placement Year Assignment.gdoc"): Operation not supported
error: unable to index file drive/MyDrive/ Placement Year Assignment.gdoc
fatal: adding files failed


In [ ]:
!ls .config

active_config  config_sentinel	configurations	gce  logs
